In [1]:
import os
import numpy as np
import pandas as pd
import urllib.request
import re
from bs4 import BeautifulSoup
import requests
from functools import reduce
from wget import download

In [2]:
URL = 'http://smass.mit.edu/catalog.php'
page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find_all('tr')[2:]
name = np.array([])
file = np.array([])
for i in range(len(table)):
    t = table[i].findAll(True, {"class":["mpname", "datalinks"]})
    if len(t) == 3:
        n = re.findall('href="(data/spex/sp.*txt)"', str(t[2]))
        if len(n)==1 and t[0].text !="":
            file = np.append(n, file)
            name = np.append(t[0].text, name)

In [4]:
asteroid_download = np.hstack((file.reshape(-1,1), name.reshape(-1,1)))

In [5]:
for i,f in enumerate(file):
    filename = os.path.basename(f)
    url = "http://smass.mit.edu/"+f
    download(url,out="VIS_NIR/")

In [6]:
asteroid_df = pd.read_csv('Data/sbdb_query_results.csv', sep=',')
asteroid_df["full_name2"] = asteroid_df["full_name"].apply( lambda x: x.split(" ")[-3])
spectral_type = np.array([])
file2 = np.array([])
name2 = np.array([])
for i in range(asteroid_df.shape[0]):
    for j in range(asteroid_download.shape[0]):
        if asteroid_df.full_name2[i] == asteroid_download[j,1]:
            spectral_type = np.append(asteroid_df.spec_B[i], spectral_type)
            name2 = np.append(asteroid_df.full_name2[i], name2)
            file2 = np.append(asteroid_download[j, 0], file2)
asteroid_df_final = pd.DataFrame({'nombre': name2, 'smass': spectral_type, 'file': file2})
asteroid_df_final['file'] = asteroid_df_final.file.apply(lambda x: x.split('/')[-1])
asteroid_df_final.head()

,nombre,smass,file
0,Abehiroshi,V,a005379.sp51.txt
1,Senrikyu,B,a005330.sp290n1.txt
2,Burbine,S,a005159.sp201n1.txt
3,Robinson,Sr,a003819.sp84.txt
4,Paris,T,a003317.sp50.txt


In [7]:
asteroid_df_final.to_csv("Data/asteroid_df_final.txt",sep="\t",index=None)